In [1]:
import re
from typing import Tuple
from llama_index.core.workflow import Event
from typing import List, Dict
from llama_index.core.workflow import (
    Context,
    Workflow,
    StartEvent,
    StopEvent,
    Event,
    step,
)

In [2]:
solution = """ 
First, compute the sum of 3 and 4, resulting in [FUNC add(3, 4) = y1]. 
Next, multiply the result by 4, resulting in [FUNC multiply(y1, 4) = y2]. 
Finally, subtract 8 from the result, resulting in [FUNC add(y2, -8) = y3].
"""

In [3]:
func_calls = re.findall(r"\[FUNC (\w+)\((.*?)\) = (\w+)\]", solution)

placeholders = set()
for match in re.finditer(r"\[FUNC (\w+)\((.*?)\) = (\w+)\]", solution):
    placeholders.add(match.group(3))

In [4]:
func_calls

[('add', '3, 4', 'y1'), ('multiply', 'y1, 4', 'y2'), ('add', 'y2, -8', 'y3')]

In [5]:
from calculator_tool_spec import CalculatorToolSpec
tools = CalculatorToolSpec().to_tool_list()
tools_by_name = {tool.metadata.name: tool for tool in tools}
tools_by_name

{'multiply': <llama_index.core.tools.function_tool.FunctionTool at 0x16a804390>,
 'add': <llama_index.core.tools.function_tool.FunctionTool at 0x16ae8f110>,
 'subtract': <llama_index.core.tools.function_tool.FunctionTool at 0x16ae8f9d0>,
 'divide': <llama_index.core.tools.function_tool.FunctionTool at 0x16ae98310>}

In [6]:
from arithmetic_validator_tool_spec import ArithmeticValidatorToolSpec
tools = ArithmeticValidatorToolSpec().to_tool_list()
validators = {tool.metadata.name: tool for tool in tools}
validators

{'eval_expression': <llama_index.core.tools.function_tool.FunctionTool at 0x16ae20890>}

In [7]:
raw_tool_output = await tools_by_name["add"].acall(3, 4)
raw_tool_output

ToolOutput(content='7', tool_name='add', raw_input={'args': (3, 4), 'kwargs': {}}, raw_output=7, is_error=False)

In [8]:
results = {}

for func_call in func_calls:
    func_name, raw_inputs, output_placeholder = func_call
    input_data = []
    
    for raw_input in raw_inputs.split(","):
        raw_input = raw_input.strip()
        
        try:
            raw_input = results[raw_input] if raw_input in results else raw_input
            input_data.append(int(raw_input))
        
        except ValueError as e:
            print("Input string cannot be converted to integer")
            input_data.append(raw_input)
            
    tool_output = await tools_by_name[func_name].acall(*input_data)
    
    if output_placeholder not in results:
        results[output_placeholder] = str(tool_output)

results

{'y1': '7', 'y2': '28', 'y3': '20'}

In [9]:
# events.py
from llama_index.core.tools.types import AsyncBaseTool
from llama_index.core.tools.function_tool import FunctionTool
import json

class FunctionCallEvent(Event):
    # input_data: List[str]
    func_call: Tuple[str, str, str]
    validator: AsyncBaseTool
    
    
class ValidateFunctionCall(Event):
    validator: AsyncBaseTool

In [22]:
class COAWorkFlow(Workflow):

    @step(pass_context=True)
    async def main_step(self, ctx: Context, ev: StartEvent) -> FunctionCallEvent:
        """Initialize the context and start processing function calls."""
        ctx.data["results"] = {} 
        ctx.data["tools_by_name"] = ev.tools_by_name
        ctx.data["function_calls"] = ev.func_calls
        ctx.data["validators"] = ev.validators
        
        validator = ctx.data["validators"]["eval_expression"]
        first_func_call = ctx.data["function_calls"][0]

        return FunctionCallEvent(func_call=first_func_call, validator=validator)

    @step(pass_context=True)
    async def function_call_step(self, ctx: Context, ev: FunctionCallEvent) -> StopEvent:
        """Execute function call"""
        func_name, raw_inputs, output_placeholder = ev.func_call
        tools_by_name = ctx.data["tools_by_name"]
        results = ctx.data["results"]
        
        input_data = []
        
        for raw_input in raw_inputs.split(","):
            raw_input = raw_input.strip()
            
            try:
                raw_input = results[raw_input] if raw_input in results else raw_input
                input_data.append(int(raw_input))
            
            except ValueError:
                print("Input string cannot be converted to integer")
                input_data.append(raw_input)
        
        tool_output = await tools_by_name[func_name].acall(*input_data)
        
        if output_placeholder not in results:
            results[output_placeholder] = str(tool_output)
            
        input_data.append(func_name)
        print(input_data)
        
        if ev.validator is not None:
            validator = ev.validator
            validator_output = await validator.acall(*input_data)
            
            if validator_output == str(tool_output):
                # return ValidationResult(validator_output) 
                pass

        return StopEvent(f"This is the result of the final function {results}")


In [23]:
w = COAWorkFlow()
await w.run(tools_by_name=tools_by_name, func_calls=func_calls, validators=validators )

[3, 4, 'add']
Validator Output:  7


"This is the result of the first function {'y1': '7'}"